#### Objective : 
The objective of this case study is to demonstrate how Artificial Neural Networks can be leveraged for classsification and how 
tuning an Artificial Neural Network can lead to an improvement in accuracy of predictions on either class.

#### Approach :
The dataset we have is infested with a high degreee of class imbalance. There are 39922 instaces that belong to classs 'no' while there are 5289 instances that belong to class 'yes'. Training the classification model on a dataset composed of samples drawn from such a dataset is likely to predict the classes belonging to the abundant class with greater correctness in comparision with oinstances that belong to the scarce class. We thus train the classification model on a dataset obtained by drawing an equal number of samples of either class and evaluating the performance of the model thus obtained on the remaining instances of the original dataset. In this way we can ensure that the performance of the model is balances on either classes i.e specificity and sensitivity values are almost identical.

#### About the Dataset :

Data Set Information :
DATA SOURCE : https://archive.ics.uci.edu/ml/machine-learning-databases/00222/

The dataset that we have has been derived from a marketing campaign run by a Portugese Banking Institution between 2008 and 2013. By training a classifier on the dataset we have, we want to evolve a model that can be used to asses the likelihood of a client subscribing to term deposit when contacted over the telephone. Clients which have a high likelihood of subscribing to the term deposit are accorded 1 and those having low likelihood are accorded 0 by the classifier.
________________________________________________________________________________________________________________________________

#### Input Features :
Bank Client data:

1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')

Related with the last contact of the current campaign :
8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

Other attributes/input features :
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

Social and Economic Context attributes/input features :
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)

#### Output Feature / Target Feature :

21 - y - has the client subscribed a term deposit? (binary: 'yes','no')


#### 1) Importing the relevant Libraries :

In [3]:
import pandas as pd
import numpy as np

#### 2) Loading the dataset :

In [4]:
bank_data=pd.read_csv('bank-full.csv',sep=';')
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [3]:
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
#checking for class imbalance:
bank_data['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

#### 3) Fetching the categorical attributes and their corresponding numerical indices from among the categorical input features "

In [5]:
#creating a dataframe of categorical input features
categorical_dataframe=bank_data.loc[:,'age':'poutcome'].select_dtypes(include=object)
#extracting the features/columns of the categorical dataframe
categorical_attributes=categorical_dataframe.columns.tolist()
#extracting the correspoding indices of categorical features
categorical_indices=[]
for attribute in categorical_attributes:
    categorical_indices.append(bank_data.columns.get_loc(attribute))
categorical_indices

[1, 2, 3, 4, 6, 7, 8, 10, 15]

#### 4) Label Encoding the categorical features :
Label Encoding is the process of assigning numerical labels to values contained within categorical input features of the dataframe. Label Encoding is performed inorder to facilitate the application of predictive mathematical models such as Logistic Regression, Support Vector Machines, Naive-Baye's etc, to those datasets which contain categorical/non-numerical data.

In [8]:
#label encoding the categorical attributes:
from sklearn.preprocessing import LabelEncoder
encoder_object=LabelEncoder()
for attribute in categorical_attributes:
    bank_data.loc[:,attribute]=encoder_object.fit_transform(bank_data.loc[:,attribute])
bank_data.head()

#label encoding the target feature:
bank_data.loc[:,'y']=encoder_object.fit_transform(bank_data.loc[:,'y'])

#### 5) OneHotEncoding the categorical features :
In order to facilitate the application of mathamatical models to datasets, merely assigning numerical labels to categorical attributes is simply not enough. One must remember that the assigned numerical labels are not related to each other in an ordinal sense, therefore we use a technique called 'OneHotEncoding' which, what basically does is, the following :

A column representing a categorical attribute is split into multiple columns such that we have new columns equal to the number of all the numerical labels used for encoding the values contained within the column under consideration. Inorder to expand upon what has just been stated, consider the following, the column of the dataframe named 'job' contains 41118 values, these 41118 values have been assigned numerical labels using integers from 0 to 12 i.e 13 integers. We will now split the 'job' column into 13 columns and each of the columns will represent an integer from 0 to 12.

For a particular observation (row index) if the job is encoded with a label '3', it will reflect in the newly created columns in the following way, the column that reprsents label '3' will be assigned 1 whereas rest of the columns will be assigned '0' and so on. This holds true for all the encoded categorical columns.

To sum up 'OneHotEncoding' can be described as the process of assigning a binary sequence of a particular 'length' to each value conatined within a 'LabelEncoded' attribute. The 'length'of the binary sequence is equal to the number of numerical labels used to represent the different values contained within a categorical column.

In [9]:
#onehotencoding:
from sklearn.preprocessing import OneHotEncoder
#while instantiating an object of the OneHotEncoder class we pass the indices of categorical features
encoder_object2=OneHotEncoder(categorical_features=categorical_indices)
bank_data=encoder_object2.fit_transform(bank_data).toarray()
bank_data=pd.DataFrame(data=bank_data)

#### 6) Checking the number of splits rendered to each categorical attribute/feature :

In [12]:
split_df=pd.DataFrame(data={'feature':categorical_attributes,'indices':categorical_indices,'splits':encoder_object2.n_values_})
split_df

,feature,indices,splits
0,job,1,12
1,marital,2,3
2,education,3,4
3,default,4,2
4,housing,6,2
5,loan,7,2
6,contact,8,3
7,month,10,12
8,poutcome,15,4


#### 7) Checking the transformed dataset:

In [7]:
bank_data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,58.0,2143.0,5.0,261.0,1.0,-1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,44.0,29.0,5.0,151.0,1.0,-1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,33.0,2.0,5.0,76.0,1.0,-1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,47.0,1506.0,5.0,92.0,1.0,-1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,33.0,1.0,5.0,198.0,1.0,-1.0,0.0,0.0


In [8]:
positive_data=bank_data[bank_data[51]==1].sample(n=3000,replace=False)
negative_data=bank_data[bank_data[51]==0].sample(n=3000,replace=False)
training_data=pd.concat([positive_data,negative_data])
training_data=training_data.reindex(np.random.permutation(training_data.index))

#removing the indices from the main dataset:
testing_data=bank_data.drop(training_data.index)

In [9]:
from sklearn.preprocessing import StandardScaler
standardizer=StandardScaler()
training_data.iloc[:,0:51]=standardizer.fit_transform(training_data.iloc[:,0:51])
testing_data.iloc[:,0:51]=standardizer.transform(testing_data.iloc[:,0:51])

In [10]:
#applying pca:
from sklearn.decomposition import PCA
pca_object=PCA(0.85)

training_input=training_data.iloc[:,0:51]
testing_input=testing_data.iloc[:,0:51]

training_input=pca_object.fit_transform(training_input)
testing_input=pca_object.transform(testing_input)

print('number of components :',pca_object.n_components_)

number of components : 30


In [11]:
X_train=pd.DataFrame(data=training_input)
X_test=pd.DataFrame(data=testing_input)
Y_train=training_data[51].values
Y_test=testing_data[51].values

In [12]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.241144,-1.019349,0.488315,0.776239,-0.965468,-0.053328,0.207850,-1.665873,-0.510022,-2.052039,...,1.155946,1.319015,-1.581708,0.096150,-3.584529,2.028250,0.999541,0.812986,0.469430,-1.714143
1,2.190469,1.520604,2.408239,-1.550920,0.121868,1.177971,-0.913854,0.881517,0.449382,2.893597,...,-0.948987,-0.492823,0.842918,1.625071,2.487433,-1.528100,0.063520,2.649644,-1.789326,-1.078712
2,-0.272051,0.055051,-0.993262,2.053624,3.310269,-1.480998,-1.864767,-0.983613,-0.660432,-1.994892,...,-0.043002,0.145348,-1.395787,0.287606,0.657211,0.055290,1.205400,-0.549031,-0.796887,-0.166154
3,-0.050385,-1.090903,-0.570913,0.149118,2.553924,-1.303054,1.191041,0.066234,0.602685,0.346460,...,0.114949,0.493352,-0.360920,-0.456799,-0.015792,-0.566415,-0.579519,-0.038638,-0.240358,0.868097
4,-0.122800,-0.470365,-2.671357,-1.641579,1.518754,0.187383,-0.720655,-0.584875,-0.116725,-0.999610,...,-0.472270,0.678100,-0.130358,-0.407093,0.109686,-0.479669,-0.624707,-0.691552,-0.142707,0.421132


In [13]:
from sklearn.svm import SVC
svm_clf=SVC()
svm_clf.fit(X_train,Y_train)
Y_pred=svm_clf.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,Y_pred))
print(100*accuracy_score(Y_test,Y_pred))

[[30619  6303]
 [  342  1947]]
83.05322486037082


In [58]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [15]:
#initializing the ANN:
classifier=Sequential() #instantiating an object of the sequential class
input_layer=Dense(units=40,kernel_initializer='uniform',activation='relu',input_dim=30) #adding hidden layers
classifier.add(input_layer)
for count in range(3):#adding hidden layers
    hidden_layer=Dense(units=40,kernel_initializer='uniform',activation='relu')
    classifier.add(hidden_layer)
    classifier.add(Dropout(rate=0.15))
output_layer=Dense(units=1,kernel_initializer='uniform',activation='sigmoid')# adding output layer
classifier.add(output_layer)
classifier.layers

#compiling the neural network:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
 classifier.fit(X_train,Y_train,batch_size=100,epochs=100)

Epoch 1/100
6000/6000 [==============================] - 0s 62us/step - loss: 0.6678 - acc: 0.6633
Epoch 2/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.4248 - acc: 0.8192
Epoch 3/100
6000/6000 [==============================] - 0s 16us/step - loss: 0.3907 - acc: 0.8370
Epoch 4/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.3733 - acc: 0.8422
Epoch 5/100
6000/6000 [==============================] - 0s 16us/step - loss: 0.3663 - acc: 0.8497
Epoch 6/100
6000/6000 [==============================] - 0s 16us/step - loss: 0.3582 - acc: 0.8547
Epoch 7/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.3491 - acc: 0.8592
Epoch 8/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.3434 - acc: 0.8610
Epoch 9/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.3395 - acc: 0.8627
Epoch 10/100
6000/6000 [==============================] - 0s 15us/step - loss: 0.3330 - acc: 0.8682
Epoch 11/

In [17]:
from sklearn.preprocessing import binarize
Y_pred=classifier.predict(X_test)
Y_pred = binarize(Y_pred,threshold=0.50)

In [18]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,Y_pred))
print(100*accuracy_score(Y_test,Y_pred))

[[30295  6627]
 [  335  1954]]
82.244778251001


In [19]:
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score
def build_classifier():
    classifier=Sequential()
    input_layer=Dense(units=40,kernel_initializer='uniform',activation='relu',input_dim=30) #adding hidden layers
    classifier.add(input_layer)
    for count in range(3):
        hidden_layer=Dense(units=40,kernel_initializer='uniform',activation='relu')
        classifier.add(hidden_layer)
    output_layer=Dense(units=1,kernel_initializer='uniform',activation='sigmoid')# adding output layer
    classifier.add(output_layer)
    classifier.layers
    classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

classifier=KerasClassifier(build_classifier,batch_size=50,epochs=100)
accuracies=cross_val_score(estimator=classifier,X=X_train,y=Y_train,cv=5)    

Epoch 1/100
4800/4800 [==============================] - 0s 70us/step - loss: 0.5924 - acc: 0.7329
Epoch 2/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.3946 - acc: 0.8346
Epoch 3/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3763 - acc: 0.8427
Epoch 4/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3673 - acc: 0.8477
Epoch 5/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3553 - acc: 0.8565
Epoch 6/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3467 - acc: 0.8548
Epoch 7/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.3407 - acc: 0.8602
Epoch 8/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3341 - acc: 0.8642
Epoch 9/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.3280 - acc: 0.8677
Epoch 10/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.3219 - acc: 0.8690
Epoch 11/

4800/4800 [==============================] - 0s 18us/step - loss: 0.2029 - acc: 0.9240
Epoch 62/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.2004 - acc: 0.9248
Epoch 63/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1973 - acc: 0.9271
Epoch 64/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1946 - acc: 0.9279
Epoch 65/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1935 - acc: 0.9258
Epoch 66/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1893 - acc: 0.9302
Epoch 67/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.1884 - acc: 0.9300
Epoch 68/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1843 - acc: 0.9296
Epoch 69/100
4800/4800 [==============================] - 0s 17us/step - loss: 0.1883 - acc: 0.9308
Epoch 70/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.1938 - acc: 0.9262
Epoch 71/100


4800/4800 [==============================] - 0s 20us/step - loss: 0.2870 - acc: 0.8887
Epoch 23/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.2829 - acc: 0.8892
Epoch 24/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.2799 - acc: 0.8921
Epoch 25/100
4800/4800 [==============================] - 0s 20us/step - loss: 0.2758 - acc: 0.8921
Epoch 26/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.2815 - acc: 0.8890
Epoch 27/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.2723 - acc: 0.8944
Epoch 28/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.2697 - acc: 0.8937
Epoch 29/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.2663 - acc: 0.8990
Epoch 30/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.2730 - acc: 0.8933
Epoch 31/100
4800/4800 [==============================] - 0s 18us/step - loss: 0.2644 - acc: 0.9000
Epoch 32/100


4800/4800 [==============================] - 0s 21us/step - loss: 0.1818 - acc: 0.9308
Epoch 86/100
4800/4800 [==============================] - 0s 20us/step - loss: 0.1837 - acc: 0.9283
Epoch 87/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1868 - acc: 0.9279
Epoch 88/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1782 - acc: 0.9300
Epoch 89/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1775 - acc: 0.9315: 0s - loss: 0.1743 - acc: 0.933
Epoch 90/100
4800/4800 [==============================] - 0s 20us/step - loss: 0.1711 - acc: 0.9365
Epoch 91/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1706 - acc: 0.9360
Epoch 92/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1741 - acc: 0.9340
Epoch 93/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.1722 - acc: 0.9333
Epoch 94/100
4800/4800 [==============================] - 0s 19us/step - loss: 0.

In [20]:
accuracies

array([0.82833333, 0.81999999, 0.81833334, 0.82833333, 0.79583332])

In [21]:
mean=accuracies.mean()
100*mean

81.81666612625122

In [22]:
X_train=X_train.values

In [23]:
Y_train

array([0., 0., 1., ..., 1., 1., 1.])

In [54]:
from keras.models import Sequential
from keras.layers import Dense
def neural_network(node_count,layer_count,opt):
    classifier=Sequential()
    input_layer=Dense(units=node_count,input_dim=30, activation='relu',kernel_initializer='uniform')
    classifier.add(input_layer)
    for count in range(layer_count):
        hidden_layer=Dense(units=node_count,activation='relu',kernel_initializer='uniform')
        classifier.add(hidden_layer)
    output_layer=Dense(units=1,activation='sigmoid',kernel_initializer='uniform')
    classifier.add(output_layer)
    classifier.compile(optimizer=opt,metrics=['accuracy'],loss='binary_crossentropy')
    return classifier

#importing the KerasClassifier inorder to facilitate hyperparameter tuning:
from keras.wrappers.scikit_learn import KerasClassifier
neural_classifier=KerasClassifier(build_fn=neural_network)
parameters={'node_count':[5,15,25,35],'layer_count':[3,4,5],'opt':['adam','rmsprop']}

#importing gridsearchcv:
from sklearn.model_selection import GridSearchCV
grid_object=GridSearchCV(estimator=neural_classifier,cv=5,scoring='accuracy',param_grid=parameters)
grid_object.fit(X_train,Y_train)

Epoch 1/1
4800/4800 [==============================] - 2s 505us/step - loss: 0.6932 - acc: 0.4935
Epoch 1/1
4800/4800 [==============================] - 3s 541us/step - loss: 0.6932 - acc: 0.4860
Epoch 1/1
4800/4800 [==============================] - 2s 519us/step - loss: 0.6927 - acc: 0.5190
Epoch 1/1
4800/4800 [==============================] - 2s 503us/step - loss: 0.6932 - acc: 0.5067
Epoch 1/1
4800/4800 [==============================] - 2s 514us/step - loss: 0.6931 - acc: 0.5019
Epoch 1/1
4800/4800 [==============================] - 2s 490us/step - loss: 0.6931 - acc: 0.5004
Epoch 1/1
4800/4800 [==============================] - 2s 496us/step - loss: 0.6932 - acc: 0.4938
Epoch 1/1
4800/4800 [==============================] - 2s 502us/step - loss: 0.6932 - acc: 0.4973
Epoch 1/1
4800/4800 [==============================] - 2s 508us/step - loss: 0.6932 - acc: 0.4912
Epoch 1/1
4800/4800 [==============================] - 2s 511us/step - loss: 0.6932 - acc: 0.4931
Epoch 1/1
4800/4800 

GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000025C8DAA37B8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'node_count': [5, 15, 25, 35], 'layer_count': [3, 4, 5], 'opt': ['adam', 'rmsprop']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [55]:
grid_object.best_params_

{'layer_count': 3, 'node_count': 25, 'opt': 'adam'}

In [56]:
classifier=Sequential()
input_layer=Dense(units=node_count,input_dim=30, activation='relu',kernel_initializer='uniform')
classifier.add(input_layer)
for count in range(layer_count):
    hidden_layer=Dense(units=node_count,activation='relu',kernel_initializer='uniform')
    classifier.add(hidden_layer)
output_layer=Dense(units=1,activation='sigmoid',kernel_initializer='uniform')
classifier.add(output_layer)
classifier.compile(optimizer=opt,metrics=['accuracy'],loss='binary_crossentropy')

Epoch 1/100
6000/6000 [==============================] - 10s 2ms/step - loss: 0.5929 - acc: 0.7415
Epoch 2/100
6000/6000 [==============================] - 1s 199us/step - loss: 0.4075 - acc: 0.8258
Epoch 3/100
6000/6000 [==============================] - 1s 197us/step - loss: 0.3900 - acc: 0.8333
Epoch 4/100
6000/6000 [==============================] - 1s 212us/step - loss: 0.3786 - acc: 0.8392
Epoch 5/100
6000/6000 [==============================] - 1s 229us/step - loss: 0.3695 - acc: 0.8438
Epoch 6/100
6000/6000 [==============================] - 1s 235us/step - loss: 0.3620 - acc: 0.8492
Epoch 7/100
6000/6000 [==============================] - 1s 231us/step - loss: 0.3547 - acc: 0.8482
Epoch 8/100
6000/6000 [==============================] - 1s 225us/step - loss: 0.3481 - acc: 0.8562
Epoch 9/100
6000/6000 [==============================] - 1s 218us/step - loss: 0.3421 - acc: 0.8585
Epoch 10/100
6000/6000 [==============================] - 1s 218us/step - loss: 0.3384 - acc: 0.8648


6000/6000 [==============================] - 1s 224us/step - loss: 0.2621 - acc: 0.8962
Epoch 83/100
6000/6000 [==============================] - 1s 224us/step - loss: 0.2608 - acc: 0.8965
Epoch 84/100
6000/6000 [==============================] - 1s 217us/step - loss: 0.2597 - acc: 0.8993
Epoch 85/100
6000/6000 [==============================] - 1s 222us/step - loss: 0.2599 - acc: 0.8957
Epoch 86/100
6000/6000 [==============================] - 1s 207us/step - loss: 0.2585 - acc: 0.8998
Epoch 87/100
6000/6000 [==============================] - 1s 224us/step - loss: 0.2580 - acc: 0.8978
Epoch 88/100
6000/6000 [==============================] - 1s 211us/step - loss: 0.2588 - acc: 0.8970
Epoch 89/100
6000/6000 [==============================] - 1s 210us/step - loss: 0.2555 - acc: 0.9007
Epoch 90/100
6000/6000 [==============================] - 1s 215us/step - loss: 0.2591 - acc: 0.8953
Epoch 91/100
6000/6000 [==============================] - 1s 211us/step - loss: 0.2561 - acc: 0.8980
Epo

In [57]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,Y_pred))
print(100*accuracy_score(Y_test,Y_pred))

[[30824  6098]
 [  383  1906]]
83.47147484124353
